![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_Bart.ipynb)

# Import ONNX BartTransformer models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in `Spark NLP 5.0.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- The BartTransformer model was introduced in `Spark NLP 5.1.0 and requires Spark version 3.4.1 and up.`
- Official models are supported, but not all custom models may work.

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.2`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.48.2 optimum onnx

- Hugging Face has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to export ONNX models directly via the CLI with `!optimum-cli export onnx --model MODEL_NAME EXPORT_PATH`.
- We'll use the [sshleifer/distilbart-xsum-12-6](https://huggingface.co/sshleifer/distilbart-xsum-12-6) model from HuggingFace as an example and export it with the `optimum-cli`.

In [2]:
MODEL_NAME = "sshleifer/distilbart-xsum-12-6"
EXPORT_PATH = f"export_onnx/{MODEL_NAME}"

!optimum-cli export onnx --model {MODEL_NAME} {EXPORT_PATH}

2025-06-08 13:33:39.346178: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749389619.371243    2003 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749389619.379603    2003 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749389619.402600    2003 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749389619.402646    2003 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749389619.402651    2003 computation_placer.cc:177] computation placer alr

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 3152796
-rw-r--r-- 1 root root      1687 Jun  8 13:34 config.json
-rw-r--r-- 1 root root 819866018 Jun  8 13:36 decoder_model_merged.onnx
-rw-r--r-- 1 root root 819603498 Jun  8 13:35 decoder_model.onnx
-rw-r--r-- 1 root root 769174126 Jun  8 13:35 decoder_with_past_model.onnx
-rw-r--r-- 1 root root 814962649 Jun  8 13:34 encoder_model.onnx
-rw-r--r-- 1 root root       329 Jun  8 13:34 generation_config.json
-rw-r--r-- 1 root root    456318 Jun  8 13:34 merges.txt
-rw-r--r-- 1 root root       957 Jun  8 13:34 special_tokens_map.json
-rw-r--r-- 1 root root      1243 Jun  8 13:34 tokenizer_config.json
-rw-r--r-- 1 root root   3558642 Jun  8 13:34 tokenizer.json
-rw-r--r-- 1 root root    798293 Jun  8 13:34 vocab.json


- We have to move additional model assets into a seperate folder, so that Spark NLP can load it properly.

In [4]:
! mkdir -p {EXPORT_PATH}/assets
! mv -t {EXPORT_PATH}/assets {EXPORT_PATH}/*.json {EXPORT_PATH}/*.txt

import json
output_json = json.load(open(f"{EXPORT_PATH}/assets/vocab.json"))

with open(f"{EXPORT_PATH}/assets/vocab.txt", "w") as f:
    for key in output_json.keys():
        print(key, file=f)

In [5]:
!ls -l {EXPORT_PATH}/assets

total 5120
-rw-r--r-- 1 root root    1687 Jun  8 13:34 config.json
-rw-r--r-- 1 root root     329 Jun  8 13:34 generation_config.json
-rw-r--r-- 1 root root  456318 Jun  8 13:34 merges.txt
-rw-r--r-- 1 root root     957 Jun  8 13:34 special_tokens_map.json
-rw-r--r-- 1 root root    1243 Jun  8 13:34 tokenizer_config.json
-rw-r--r-- 1 root root 3558642 Jun  8 13:34 tokenizer.json
-rw-r--r-- 1 root root  798293 Jun  8 13:34 vocab.json
-rw-r--r-- 1 root root  407065 Jun  8 13:36 vocab.txt


## Import and Save BartTransformer in Spark NLP

Let's install and setup Spark NLP in Google Colab. For this example, we'll use specific versions of `pyspark` and `spark-nlp` that we've already tested with this transformer model to make sure everything runs smoothly:

If you prefer to use the latest versions, feel free to run:

`!pip install -q pyspark spark-nlp`

Just keep in mind that newer versions might have some changes, so you may need to tweak your code a bit if anything breaks.

In [6]:
! pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 30.2 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `BartTransformer` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `BartTransformer` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [8]:
from sparknlp.annotator import BartTransformer

bart = (
    BartTransformer.loadSavedModel(f"{EXPORT_PATH}", spark)
    .setInputCols("documents")
    .setOutputCol("generation")
    .setTask("summarize:")
)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
bart.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your ONNX BartTransformer model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {MODEL_NAME}_spark_nlp

total 1596520
-rw-r--r-- 1 root root 819728713 Jun  8 13:39 decoder_model.onnx
-rw-r--r-- 1 root root 815087154 Jun  8 13:39 encoder_model.onnx
drwxr-xr-x 4 root root      4096 Jun  8 13:38 fields
drwxr-xr-x 2 root root      4096 Jun  8 13:38 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny BartTransformer model 😊

In [12]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import BartTransformer
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")

bart = BartTransformer.load(f"{MODEL_NAME}_spark_nlp")

pipeline = Pipeline(stages=[
    document_assembler,
    bart
])

example = spark.createDataFrame([[
    """PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions.
    The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were
    scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."""
]], ["text"])

result = pipeline.fit(example).transform(example)
result.select("text", "generation.result").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                                                                               |result                                                                       |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

That's it! You can now go wild and use hundreds of BartTransformer models from HuggingFace 🤗 in Spark NLP 🚀
